In [ ]:
#Read in PMID csv file created through
path = "/Users/manatee_prime/Desktop/NCA/"

citation = [[],[],[]]
scrape = [[],[]]

f = open(path + 'NCA_citations.csv','r')
for line in f:
    if line.split('|')[0] != 'NCA':
        citation[0].append(line.split('|')[0])
        citation[1].append(line.split('|')[1])
        citation[2].append(line.split('|')[2])
        scrape[0].append(line.split('|')[3])
        scrape[1].append(line.split('|')[4].rstrip())

print(len(citation[0]),len(citation[1]),len(citation[2]),len(scrape[0]),len(scrape[1]))

In [ ]:
#Note: need way to handle and remove the "page X of \n Y" from the documents as it can ruin a good citation
import os
from datetime import datetime

path = "/Users/manatee_prime/Desktop/NCA/"

###Create list of only txt files for reading###
dir_nca = os.listdir(path) #get all file names on path

dir_nca_txt = [] #create new list to hold only the .txt files
for item in dir_nca: 
    if ".txt" in item: #include only txt files
        dir_nca_txt.append(item)
#dir_nca = dir_nca[1:10]

###Global variables for containing ALL NCA info###
citation = [[],[],[]] #0 = memo indicating origin doc, #1 = date, #2 = citation
for nca in dir_nca_txt:
    f = open(path + nca,'r')
    memo = False
    date = False
    bib = False
    tmpbuff = []
    for line in f: 
        ###Grab document topic for later comparison###
        if memo == False and line != "\n":
            tmpbuff.append(line.rstrip())
        if memo == False and line == "\n":
            memo = " ".join(tmpbuff) #memo will be used to code the citation below
        
        ###Grab date information###
        if line.split(" ")[0] == 'Date:' or line.split(" ")[0] == 'DATE:':
            if date == False: 
                date = True
                continue
            
        if date == True and line != '\n':
            tmpdate = line.replace(" ","").rstrip()
            if len(tmpdate.split(",")[0]) > 4:
                try:
                    date = datetime.strptime(tmpdate, '%B%d,%Y')
                except ValueError:
                    print("DATETIME FAULT!",nca,'  ',date)
                    break
            if len(tmpdate.split(",")[0]) == 4:
                try:
                    date = datetime.strptime(tmpdate, '%b%d,%Y')
                except ValueError:
                    print("DATETIME FAULT! ",tmpdate,'\t',nca)
                    break
        
        ###Grab author information###
        #Could update to include CCSQ author information
        
        ###Grab bibliometric information###
        if bib == True:
            if line != "\n" and line.split(" ")[0] != "Created":
                if "\x0c" in line:
                    tmpbuff.append(line.split("\x0c")[1].rstrip())
                else:
                    tmpbuff.append(line.rstrip())
            if line == "\n" and len(tmpbuff) > 0:
                citation[0].append(memo) #code citation with origin document
                if date == False or date == True:
                    print("BOOLEAN FAULT! ",nca,'  ',date)
                citation[1].append(date) #code citation with date of citing document
                
                tmpbuff = " ".join(tmpbuff)
                citation[2].append(tmpbuff) #append citation to final list
                tmpbuff = []
        if line == "Bibliography\n":
            bib = True
    f.close()
    
print(len(citation[0]),len(citation[1]),len(citation[2]))

In [ ]:
#This cell creates a stopwords set for comparison to putative title sections
#Additional terms have been added to the stopwords dictionary to try and capture common science terms
#NOTE: It would be interesting to look for a library of the most common words in science!

#Run this cell once to get the stop variable necessary to get pmids

import nltk
from nltk.corpus import stopwords

###Set up stopwords for evaluating title strings###
stop = set(stopwords.words('english'))
b = set([])
for x in ['test','tests','tested','testing','study','studies','studied','studying','clinic','clinical','review',
          'investigation','investigate','investigating','investigated','patient','patients','measure','measured',
          'measuring','treatment','treat','treating','treats','treated','therapy','therapies','therapeutic',
         'effect','effects','effected','effecting','disease','condition']:
    stop.add(x) #add the above terms to the stop list
for x in stop:
    if len(x) < 2:
        b.add(x) #make new set of all elements in stop that are 1 letter long
stop -= b #remove intersection of stop and b from stop (ie stop = stop - b)

print(stop)

In [ ]:
#This block converts the free text citations into PMIDs where possible
#NOTE: This script modifies the citation list made above so will need to rerun that script BEFORE rerun this one
#This version uses titles as they are the easiest to identify but using the journal/vol/pages could be better

from difflib import SequenceMatcher
import re
import requests
import urllib.request
import time
from bs4 import BeautifulSoup


#Create function to find similarity
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

#tester = citation[2][0:50]

scrape = [[],[]] #0 = pmid #,'broken','url',etc. #1 = simscore value
ct = 0 #used to keep track of iteration number
ct2 = 0

for i in range(len(citation[2])):
#for i in range(2000):
    ct += 1
    if ct % 300 == 0:
        print(ct,' completed of ',len(citation[2]))
    title = ""
    jrndat = "" #currently not in use but could use #letters/#words with re.sub to id the journal,vol,page info
    separator = "."
    found = False

    ######PARSING PMIDS FROM CITATIONS W PMIDS######
    if 'PMID' in citation[2][i]:
        junk = re.sub('[^0-9]','',citation[2][i].split('PMID')[1])
        ###Manually identified corrections###
        #Note - PMIDs have max 8 digits and most have 8, making that assumption is generally ok
        if junk == '233864216': #correction
            junk = '23864216'
        if 'A-Derogatis LR, Lipman RS, Rickels K, Uhlenhuth EH, Covi L' in citation[2][i]:
            junk = '4808738'
        if 'Hoenig J, Kenna JC, Youd A. Surgical treatment for transsexualism' in citation[2][i]:
            junk = '5096332'
        if 'Hoenig J, Kenna J. Epidemiological aspects of transsexualism' in citation[2][i]:
            junk = '4705331'
        if 'Juckett, G. Cross-Cultural Medicine. Am Fam Physician.' in citation[2][i]:
            junk = '16342851'
        if junk == '283545633':
            junk = '28545633'
        if junk == '162665560':
            junk = '16266560'
        if junk == '855872510':
            junk = '8558725'
        if 'Nguyen N and Holodniy M.  HIV infection in the elderly' in junk:
            junk = '18982916'
        ###Parse PMIDS###
        if len(junk) > 8:
            junk = junk[0:8]
        scrape[0].append(junk)
        scrape[1].append('NA')
        continue
    ######END BLOCK######
    
    ######PARSING CITATIONS######
    ###Remove urls from citation pool###
    if 'http://' in citation[2][i] or 'https://' in citation[2][i] or 'www.' in citation[2][i]: #ignore all the website data
        scrape[0].append('url')
        scrape[1].append('NA')
        continue
    
    ###Look for citations with quotes###
    if citation[2][i].count('\'') != 2 and citation[2][i].count('\'') != 0: #Here to catch crazy weird citations
        scrape[0].append('q broken')
        scrape[1].append('NA')
        continue
    if citation[2][i].count('\'') == 2: 
        if citation[2][i].split('\'')[1].count(' ') > 2:
            separator = '\''

    ###Look for title, journal/date using separator###
    tmpq = citation[2][i].lower().replace('et al','').split(separator) #think about using the et al as a split!
        
    #Find the title using length and stopwords
    title = [re.sub('[^A-z]',' ',x) for x in tmpq] #get rid of numbers to ensure length best measure
    test = [1,0,0] #0 = index in title, 1 = number of words (split on ' '), 2 = # stop words
    
    for x in range(len(title)):
        comp = list(filter(None,title[x].split(' '))) #create list and remove empties from it
        complst = [x, len(comp), len(stop & set(comp))]
        if complst[1] + 5*complst[2] > test[1] + 5*test[2]: #this weighting gave optimal # results
            test = complst
    if test == [1,0,0] or len(tmpq[test[0]].split(' ')) < 3:
        scrape[0].append('broken')
        scrape[1].append('NA')
        continue
    
    title = tmpq[test[0]] #set title to the best term in the original tmpq list
    title = re.sub('[^A-z|0-9|\s|\:|\-|,|;]','',title)
    
    ###Search for PMID using EUtils and query###
    skey = 'dcff69acdc62008c90b1354b0ac141dc0808' #api key to get more queries to ncbi eutils
    tmpmid = []
    
    #Search for PMID
    url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term={0}&api_key={1}'.format(title,skey)
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'html.parser') #try and run xml.parser to see if it has one
    content = soup.find_all('id')
    if content == None:
        scrape[0].append('broken')
        scrape[1].append('NA')
        continue
    if content != None:
        tmpmid = [str(x).split('>')[1].split('<')[0] for x in content]
    if len(tmpmid) > 5:
        tmpmid = tmpmid[0:5]
        ct2 += 1
    
    #Check if PMID correct
    for putative in tmpmid:
        time.sleep(0.1)
        tstitle = ''
        url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&id={0}&api_key={1}'.format(putative,skey) #fetch summary for dbck
        response = requests.get(url)
        soup = BeautifulSoup(response.text,'html.parser')
        
        content = str(soup.find('item', attrs={'name':"Title"})).split('>')[1].split('<')[0]
        tstitle = re.sub('[^A-z|0-9|\s|:|\-|,|;]','',content).lower()
        pages = str(soup.find('item',attrs={'name':"Pages"})).split('>')[1].split('<')[0]
        vol = str(soup.find('item',attrs={'name':"Volume"})).split('>')[1].split('<')[0]
        if pages == '':
            pages = '{FAKE!}'
        if vol == '':
            vol = '{FAKE!}'
        
        #Commented out: jaccard distance using n-grams
        #n1 = set(nltk.ngrams(tstitle,n=5))
        #n2 = set(nltk.ngrams(title,n=5)) 
        #if 1 - nltk.jaccard_distance(n1,n2) > 0.8:
        if similar(tstitle,title) > 0.9: #the false positive rate here is negligible so take anything that hits
            scrape[0].append(putative)
            #scrape[0].append(content) #false positive chk
            scrape[1].append(similar(tstitle,title))
            found = True
            break
        #Commented out: jaccard distance using n-grams
        #if 1 - nltk.jaccard_distance(n1,n2) > 0.5 and pages in citation[2][i] and vol in citation[2][i]:
        if similar(tstitle,title) > 0.5 and pages in citation[2][i] and vol in citation[2][i]: #o-cutoff = 0.667
            scrape[0].append(putative)
            #scrape[0].append(content) #false positive chk
            scrape[1].append(similar(tstitle,title))
            found = True
            break    
    
    #If everything failed, keep tack of citation anyways
    if found == False:
        scrape[0].append('failed')
        scrape[1].append('NA')
    ######END BLOCK######
        
print(len(scrape[0]),len(scrape[1]),scrape[0].count('failed') + scrape[0].count('broken') +
      scrape[0].count('q broken') + scrape[0].count('url'))

In [ ]:
######SCRATCH FOR WORKING WITH PMID FINDING CODE######
'''
s1 = 'Embolism treated with mitral clip and cardiac stents in situ'
s2 = 'Embolism treated with mitral clip and cardiac stents in place'
w1 = set(s1)
w2 = set(s2)
n1 = set(nltk.ngrams(s1,n=3))
n2 = set(nltk.ngrams(s2,n=3))

print(similar(s1,s2),1 - nltk.jaccard_distance(w1,w2),1 - nltk.jaccard_distance(n1,n2),
      1 - nltk.edit_distance(s1,s2)/max(len(s1),len(s2)))'''

for i in range(len(scrape[0])):
    if scrape[1][i] != 'NA':
        if 0.8 < scrape[1][i] < 0.95:
        #if 0.95 < scrape[1][i]:
            print(citation[2][i]+'\n',scrape[0][i]+'\n',scrape[1][i],'\n')

In [ ]:
#Simple evaluation stats for testing false positives

from matplotlib import pyplot as plt
import collections

print('Total number of PMIDS: ',len(hdat),'\n',
      'Unique PMIDs: ', len(collections.Counter(hdat)))

ct = 0 
hdat = []
for dat in scrape[1]:
    if dat != 'NA':
        ct += 1
        hdat.append(float(dat)) #float needed when importing scrape[1] data from file

plt.hist(hdat,50)
plt.xlim(0.5,1.0)
plt.ylim(0,100)

plt.show()
#plt.close()

In [ ]:
######WRITE OUT NCA_citations.csv FILE FOR R ANALYSIS######

import csv

#Be sure to remove delimiter from ALL indexes in the citation list!
for i in range(len(citation[2])):
    if '|' in citation[2][i]:
        citation[2][i] = citation[2][i].replace('|','')
        print(citation[2][i])

with open(path + 'NCA_citations.csv', 'w', encoding = 'utf-8') as csvfile:
    writer = csv.writer(csvfile, delimiter='|')
    writer.writerow(['NCA','Date','Text','Citation','Similarity'])
    for i in range(len(citation[0])):
        writer.writerow([citation[0][i],citation[1][i],citation[2][i],scrape[0][i],scrape[1][i]])

In [ ]:


######BEGINNING GRANT ANALYSIS CODE######


#This code returns grant information on all pmids gathered from pubmed
import collections
import requests
import urllib.request
from bs4 import BeautifulSoup
import time

#NOTE: consider adding article date information to grant scraper for accurate dating down to the day
#      use 'pubdate' to grab date information for future grant searches
grant = [[],[],[],
         [],[],[],[]] #0=originator CMS doc, 1=free txt citation, 2=citation PMID, 3=grant number, 4=grant agency, 5=country
ct = 0

pmidlist = collections.Counter(scrape[0])

for pmid in range(len(scrape[0])):
    #Counter to track number of citations tackled
    ct += 1
    if ct % 300 == 0:
        print(pmid/len(citation[2]),' completed!')
    
    try:
        int(scrape[0][pmid]) #Test if PMID as all PMIDs are integers
        
        skey = 'dcff69acdc62008c90b1354b0ac141dc0808' #api key to get more queries to ncbi eutils
        time.sleep(0.2) #setting to 0.1 throws over-query error
        url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id={0}&retmode=xml&api_key={1}".format(scrape[0][pmid],skey)
        response = requests.get(url)
        soup = BeautifulSoup(response.text,'html.parser')
        
        ###SCRAPE PUBLICATION DATE INFORMATION###
        date = soup.find_all('pubdate')
        pubdate = ''
        for item in date:
            for subitem in item: #pubdate lists in order: year, month, day
                if 'year' in str(subitem):
                    pubdate = str(subitem).split('>')[1].split('<')[0]
                if 'month' in str(subitem):
                    pubdate = pubdate + '-' + str(subitem).split('>')[1].split('<')[0]
                if 'day' in str(subitem):
                    pubdate = pubdate + '-' + str(subitem).split('>')[1].split('<')[0]
            if a.count('-') == 1:
                pubdate = pubdate + '-15' #arbitrarily set Y-M dates to Y-M-15 (middle of month)
        grant[6].append(pubdate)
        ###END BLOCK###
        
        ###SCRAPE GRANT INFORMATION###
        content = soup.find_all('grant')
        if content == None:
            grant[0].append(citation[0][pmid])
            grant[1].append(citation[2][pmid])
            grant[2].append(scrape[0][pmid])
            grant[3].append('NA')
            grant[4].append('NA')
            grant[5].append('NA')
            continue

        for item in content:
            gid = False
            agency = False
            country = False
            grant[0].append(citation[0][pmid])
            grant[1].append(citation[2][pmid])
            grant[2].append(scrape[0][pmid])
            
            for j in item:
                if 'grantid' in str(j):
                    grant[3].append(str(j).split('>')[1].split('<')[0])
                    gid = True
                    continue
                if 'agency' in str(j):
                    grant[4].append(str(j).split('>')[1].split('<')[0])
                    agency = True
                    continue
                if 'country' in str(j):
                    grant[5].append(str(j).split('>')[1].split('<')[0])
                    country = True
                    continue
            
            if gid == False:
                grant[3].append('NA')
            if agency == False:
                grant[4].append('NA')
            if country == False:
                grant[5].append('NA')
        ###END BLOCK###
                
    except ValueError: #Pass by non-PMID entries
        #grant[0].append('NA')
        #grant[1].append('NA')
        #grant[2].append('NA')
        continue

print(len(grant[0]),len(grant[1]),len(grant[2]),len(grant[3]),len(grant[4]),len(grant[5]))

In [ ]:
#Simple evaluation stats for grant data

from matplotlib import pyplot as plt
import collections

agencies = collections.Counter(grant[4])
pmids = collections.Counter(grant[2])
dat = []
for x in pmids:
    dat.append(pmids[x])
    if dat[-1] > 10:
        print(x,'\t',pmids[x])

print(agencies)
    
plt.hist(dat,50)
#plt.xlim(0,90)
#plt.ylim(0,10)

plt.show()
#plt.close()

In [ ]:
#Correct known synonyms for grant agencies
nodes = [x for x in collections.Counter(grant[4])]
for item in nodes:
    print(item)
'''
syns = ['Canadian Institutes of Health Research', 'Canadian Institute of Health Research', 'CIHR']

for i in range(len(grant[4])):
    if grant[4][i] in syns:
        grant[4][i] = 'Canadian Institutes of Health Research'
        print('corrected!')
'''

In [ ]:
#Create nodes and edges file for visNetwork in R
#Nodes file == list of all possible nodes
#Edges file == [[from],[to],[weight]], for this analysis weight == 1 unless to CMS, then it == counts
#NOTE - Have not removed redundant citations. This COULD provide a basic weighting that would be useful
#       BUT it would probably over-weight citations from similar/multi-stage NCAs

import collections

pmids = collections.Counter(grant[2])
tmpl2 = [] #this will house all the combinations of grants
#edges = [[],[],[]] #0 = origin node, 1 = destination node, 2 = edge weight

for pmid in pmids:
    tmpl = [] #re-create tmp list for each iteration
    
    #assemble all grants for a particular pmid in tmpl
    for i in range(len(grant[2])):
        if grant[2][i] == pmid and grant[4][i] != 'NA':
            tmpl.append(grant[4][i])
            
    if len(tmpl) == 0: #move on from missing grant data
        continue
    if len(tmpl) == 1: #ensure singletons still count towards CMS
        tmpl2.append(tmpl[0]+'_CMS')
        continue
        
    #join all grants funding same publication
    tmpl = sorted(tmpl) #arrange alphabetically to impose standard order
    for j in range(len(tmpl)):
        tmpl2.append(tmpl[j]+'_CMS') #each grant in tmpl[0] connects to CMS
        for k in range(j+1,len(tmpl)):
            tmpl2.append(tmpl[j]+'_'+tmpl[k]) #each grant within a pub should connect to eachother

edgect = collections.Counter(tmpl2)

In [ ]:
import csv
path = "/Users/manatee_prime/Desktop/NCA/"

#write out raw grant info file
with open(path + 'NCA_grants.csv', 'w', encoding = 'utf-8') as csvfile:
    writer = csv.writer(csvfile, delimiter='|')
    writer.writerow(['NCA','Text','PMID','Grant Number','Grant Agency','Agency Country']) #set up to match visNetwork syntax
    for i in range(len(grant[0])):
        writer.writerow([grant[0][i],grant[1][i],grant[2][i],grant[3][i],grant[4][i],grant[5][i]])
csvfile.close()

#write out edges file
with open(path + 'NCA_grantagency_ntwk_edges.csv', 'w', encoding = 'utf-8') as csvfile:
    writer = csv.writer(csvfile, delimiter='|')
    writer.writerow(['from','to','value']) #set up to match visNetwork syntax
    for item in edgect:
        writer.writerow([item.split('_')[0],item.split('_')[1],edgect[item]])
csvfile.close()

#write out nodes file
nodes = [x for x in collections.Counter(grant[4])]
with open(path + 'NCA_grantagency_ntwk_nodes.csv', 'w', encoding = 'utf-8') as csvfile:
    writer = csv.writer(csvfile, delimiter='|')
    writer.writerow(['id']) #set up to match visNetwork syntax
    writer.writerow(['CMS']) #add cms to the nodes network
    for item in nodes:
        writer.writerow([item])
csvfile.close()

In [5]:
import os
from datetime import datetime

path = "/Users/manatee_prime/Desktop/"
i = 0

f = open(path + 'fda_test.txt','r')
for line in f:
    i += 1
    if i < 300:
        print(line)
    if i > 300:
        break

Long Term Follow-Up After 

Administration of Human Gene 

Therapy Products 



Guidance for Industry 



Additional copies of this guidance are available from the Office of Communication, Outreach 

and Development (OCOD), 10903 New Hampshire Ave., Bldg. 71, Rm. 3128, Silver Spring, 

MD 20993-0002, or by calling 1-800-835-4709 or 240-402-8010, or email ocod@fda.hhs.gov, or 

from the Internet at https://www.fda.gov/vaccines-blood-biologics/guidance-compliance-

regulatory-information-biologics/biologics-guidances. 



For questions on the content of this guidance, contact OCOD at the phone numbers or email 

address listed above. 



U.S. Department of Health and Human Services 

Food and Drug Administration 

Center for Biologics Evaluation and Research 

January 2020 



 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

Contains Nonbinding Recommendations 



Table of Contents 



IV. 



V. 



C. 



D. 



B. 

C. 



I. 

II. 

III. 



INTRODUCTION